## Web Scraper using an AI *(ChatGPT)*

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page to scrape
url = "https://pokemondb.net/pokedex/all"

# Make a GET request to fetch the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Locate the table
table = soup.find('table', {'id': 'pokedex'})

# Extract table headers
headers = [header.text for header in table.find_all('th')]

# Extract table rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = row.find_all('td')
    data = [cell.text.strip() for cell in cells]
    # Add sprite image URL (bonus)
    sprite = row.find('img')['src'] if row.find('img') else None
    data.append(sprite)
    rows.append(data)

# Add "Sprite" to headers for the image column
headers.append('Sprite')

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Convert numeric columns to appropriate types
numeric_columns = ['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Save the DataFrame for further analysis
df.to_csv("pokemon_data.csv", index=False)

# Display the first few rows
print(df.head())

   #                    Name          Type  Total  HP  Attack  Defense  \
0  1               Bulbasaur  Grass Poison    318  45      49       49   
1  2                 Ivysaur  Grass Poison    405  60      62       63   
2  3                Venusaur  Grass Poison    525  80      82       83   
3  3  Venusaur Mega Venusaur  Grass Poison    625  80     100      123   
4  4              Charmander          Fire    309  39      52       43   

   Sp. Atk  Sp. Def  Speed                                             Sprite  
0       65       65     45  https://img.pokemondb.net/sprites/scarlet-viol...  
1       80       80     60  https://img.pokemondb.net/sprites/scarlet-viol...  
2      100      100     80  https://img.pokemondb.net/sprites/scarlet-viol...  
3      122      120     80  https://img.pokemondb.net/sprites/scarlet-viol...  
4       60       50     65  https://img.pokemondb.net/sprites/scarlet-viol...  


## Exploratory data analysis using an AI *(ChatGPT)*

### Strongest Pokemon

In [14]:
# Find the strongest Pokémon of each type based on 'Total' stats

strongest_pokemon = df.loc[df.groupby('Type')['Total'].idxmax()]
print("Strongest Pokémon of each type:")
print(strongest_pokemon[['Name', 'Type', 'Total']])

Strongest Pokémon of each type:
                          Name           Type  Total
167                     Pinsir            Bug    500
1095                     Lokix       Bug Dark    450
888                   Vikavolt   Bug Electric    500
896                   Ribombee      Bug Fairy    464
274   Heracross Mega Heracross   Bug Fighting    600
...                        ...            ...    ...
176                     Lapras      Water Ice    535
94                  Tentacruel   Water Poison    515
108       Slowbro Mega Slowbro  Water Psychic    590
687                 Carracosta     Water Rock    495
489                   Empoleon    Water Steel    530

[221 rows x 3 columns]


### Best Attackers

In [19]:
# Find the Pokémon with the highest Attack stats

best_attackers = df.nlargest(10, 'Attack')[['Name', 'Attack', 'Type']]
print("Top 10 Pokémon by Attack:")
print(best_attackers)

Top 10 Pokémon by Attack:
                         Name  Attack              Type
201      Mewtwo Mega Mewtwo X     190  Psychic Fighting
274  Heracross Mega Heracross     185      Bug Fighting
956                   Kartana     181       Grass Steel
473    Groudon Primal Groudon     180       Ground Fire
475    Rayquaza Mega Rayquaza     180     Dragon Flying
478       Deoxys Attack Forme     180           Psychic
545    Garchomp Mega Garchomp     170     Dragon Ground
777       Kyurem Black Kyurem     170        Dragon Ice
961   Necrozma Ultra Necrozma     167    Psychic Dragon
436      Banette Mega Banette     165             Ghost


### Averages of Stats for Each Type

In [44]:
# Calculate average stats for each primary type (ensure column name matches)

type_averages = df.groupby('Type').mean(numeric_only=True)[numeric_columns]
print("Average stats for each primary type:")
print(type_averages)

Average stats for each primary type:
                    Total         HP      Attack     Defense     Sp. Atk  \
Type                                                                       
Bug            280.280000  48.920000   48.480000   54.960000   37.560000   
Bug Dark       450.000000  71.000000  102.000000   78.000000   52.000000   
Bug Electric   422.750000  63.500000   69.000000   73.750000   88.500000   
Bug Fairy      384.000000  50.000000   50.000000   50.000000   75.000000   
Bug Fighting   562.000000  84.600000  144.200000   89.000000   71.000000   
...                   ...        ...         ...         ...         ...   
Water Ice      510.000000  90.000000   85.000000  110.000000   80.000000   
Water Poison   430.000000  61.666667   68.333333   61.666667   61.666667   
Water Psychic  479.714286  84.714286   81.714286   94.714286   88.285714   
Water Rock     446.000000  76.600000   89.200000  110.200000   58.800000   
Water Steel    530.000000  84.000000   86.000000   

###  Fastest Pokemon

In [27]:
# Find the fastest Pokémon

fastest_pokemon = df.nlargest(10, 'Speed')[['Name', 'Speed', 'Type']]
print("Top 10 Fastest Pokémon:")
print(fastest_pokemon)

Top 10 Fastest Pokémon:
                             Name  Speed            Type
1063                    Regieleki    200        Electric
480            Deoxys Speed Forme    180         Psychic
361                       Ninjask    160      Bug Flying
953                     Pheromosa    151    Bug Fighting
86         Alakazam Mega Alakazam    150         Psychic
135                     Electrode    150        Electric
136   Electrode Hisuian Electrode    150  Electric Grass
189    Aerodactyl Mega Aerodactyl    150     Rock Flying
477           Deoxys Normal Forme    150         Psychic
478           Deoxys Attack Forme    150         Psychic


### Pokemon with the highest HP

In [34]:
# Pokémon with the highest HP

most_hp = df.nlargest(1, 'HP')[['Name', 'HP', 'Type']]
print("Pokémon with the most HP:")
print(most_hp)

Pokémon with the most HP:
        Name   HP    Type
305  Blissey  255  Normal


### Display Pokemon Sprites

In [31]:
from IPython.display import Image, display

# Display sprite images for the top 5 Pokémon by Total stats
for _, row in strongest_pokemon.head(5).iterrows():
    print(row['Name'])
    display(Image(url=row['Sprite']))


Pinsir


Lokix


Vikavolt


Ribombee


Heracross Mega Heracross


## Pokedex Application with a graphical Interface using AI *(ChatGPT)*

In [62]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import requests
from io import BytesIO
from PIL import Image
import base64

# Load Pokémon dataset
df = pd.read_csv("pokemon_dataset.csv")  # Use your scraped dataset CSV

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Pokémon Viewer"

# Define app layout
app.layout = html.Div([
    html.H1("Pokémon Viewer", style={'text-align': 'center'}),
    
    # Sorting and Filtering Options
    html.Div([
        html.Label("Sort by:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='sort-dropdown',
            options=[{'label': col, 'value': col} for col in ['Attack', 'Defense', 'HP', 'Sp. Atk', 'Sp. Def', 'Speed']],
            value='Attack',
            style={'width': '200px', 'display': 'inline-block'}
        ),
        html.Button("Toggle Asc/Desc", id='sort-order-button', n_clicks=0, style={'margin-left': '10px'}),
        html.Label("Filter by Type:", style={'margin-left': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='type-dropdown',
            options=[{'label': t, 'value': t} for t in sorted(df['Type'].unique())],
            multi=True,
            style={'width': '300px', 'display': 'inline-block'}
        )
    ], style={'margin': '20px'}),
    
    # Pokémon Cards
    html.Div(id='pokemon-cards', className='d-flex flex-wrap justify-content-center'),
    
    # Footer
    html.Div("Built with Dash and Pokémon Data", style={'text-align': 'center', 'margin-top': '20px', 'font-size': '12px'})
])

# Helper function to encode image URLs as base64
def fetch_image_as_base64(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    buffer.seek(0)
    return base64.b64encode(buffer.read()).decode()

# Callback to update Pokémon cards
@app.callback(
    Output('pokemon-cards', 'children'),
    Input('sort-dropdown', 'value'),
    Input('sort-order-button', 'n_clicks'),
    Input('type-dropdown', 'value')
)
def update_pokemon_cards(sort_by, sort_order_clicks, selected_types):
    # Sort the DataFrame
    ascending = (sort_order_clicks % 2 == 0)
    sorted_df = df.sort_values(by=sort_by, ascending=ascending)

    # Filter by Type
    if selected_types:
        sorted_df = sorted_df[sorted_df['Type'].isin(selected_types)]

    # Generate Pokémon cards
    cards = []
    for _, row in sorted_df.iterrows():
        # Fetch image
        sprite_base64 = fetch_image_as_base64(row['Sprite'])

        # Create a card for each Pokémon
        card = dbc.Card(
            [
                dbc.CardImg(src=f"data:image/png;base64,{sprite_base64}", top=True, style={'height': '120px'}),
                dbc.CardBody([
                    html.H5(f"#{row['#']} {row['Name']}", className='card-title'),
                    html.Div([
                        html.Div([html.Span(f"{stat}: ", style={'font-weight': 'bold'}), row[stat]], 
                                 style={'margin-bottom': '5px'})
                        for stat in ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
                    ], style={'font-size': '12px'}),
                ])
            ],
            style={'width': '200px', 'margin': '10px'}
        )
        cards.append(card)

    return cards

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)